<a href="https://colab.research.google.com/github/Cinnamy/DocumentsClassification/blob/main/6_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Подключаем модули

In [ ]:
import sklearn
import numpy as np
import nltk
import re
import pandas as pd
import torch
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora, models
from gensim.utils import simple_preprocess

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics
from sklearn.datasets import fetch_20newsgroups
from transformers import DistilBertTokenizer, DistilBertModel

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


#Загружаем и предобрабатываем данные

In [ ]:
stopwords_en = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

articles = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=['comp.graphics', 'sci.med'])
articles_full = []
articles_full_bow = []
articles_target = [[], [], []]
articles_nouns = []
articles_nouns_bow = []
articles_nouns_and_adjs = []
articles_nouns_and_adjs_bow = []
for i in range(300):
  text = articles.data[i]
  text = re.sub(r'[^(\w\s\-)]|[\d]|[()]|-{2,}', '', text)
  text = re.sub(r'\s{2,}|[\n]', ' ', text)
  text = [lemmatizer.lemmatize(word.lower()) for word in word_tokenize(text) if word not in stopwords_en]
  if text != []:
    articles_full.append(' '.join(text))
    articles_full_bow.append(text)
    articles_target[0].append(articles.target[i])
  nouns = [word[0] for word in nltk.pos_tag(text) if word[1] == "NN"]
  if nouns != []:
    articles_nouns.append(' '.join(nouns))
    articles_nouns_bow.append(nouns)
    articles_target[1].append(articles.target[i])
  nouns_and_adjs = [word[0] for word in nltk.pos_tag(text) if word[1] == "NN" or word[1] == "JJ"]
  if nouns_and_adjs != []:
    articles_nouns_and_adjs.append(' '.join(nouns_and_adjs))
    articles_nouns_and_adjs_bow.append(nouns_and_adjs)
    articles_target[2].append(articles.target[i])

#Векторизация

##TF-IDF

In [ ]:
vectorizer = TfidfVectorizer()
X_full_TF = vectorizer.fit_transform(articles_full)
X_nouns_TF = vectorizer.fit_transform(articles_nouns)
X_nouns_and_adjs_TF = vectorizer.fit_transform(articles_nouns_and_adjs)

##DistilBERT embeddings

In [ ]:
model_name = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
articles_full_tokens = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in articles_full]
articles_nouns_tokens = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in articles_nouns]
articles_nouns_adjs_tokens = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in articles_nouns_and_adjs]

X_full_DBERT = []
with torch.no_grad():
  for phrase_tokens in articles_full_tokens:
    outputs = model(**phrase_tokens)
    X_full_DBERT.append(np.array(outputs.last_hidden_state.mean(dim=1).flatten()))

X_nouns_DBERT = []
with torch.no_grad():
  for phrase_tokens in articles_nouns_tokens:
    outputs = model(**phrase_tokens)
    X_nouns_DBERT.append(np.array(outputs.last_hidden_state.mean(dim=1).flatten()))

X_nouns_and_adjs_DBERT = []
with torch.no_grad():
  for phrase_tokens in articles_nouns_adjs_tokens:
    outputs = model(**phrase_tokens)
    X_nouns_and_adjs_DBERT.append(np.array(outputs.last_hidden_state.mean(dim=1).flatten()))

##LDA

In [ ]:
dictionary_full = corpora.Dictionary(articles_full_bow)
dictionary_nouns = corpora.Dictionary(articles_nouns_bow)
dictionary_nouns_and_adjs = corpora.Dictionary(articles_nouns_and_adjs_bow)

bow_full = [dictionary_full.doc2bow(doc) for doc in articles_full_bow]
bow_nouns = [dictionary_nouns.doc2bow(doc) for doc in articles_nouns_bow]
bow_nouns_and_adjs = [dictionary_nouns_and_adjs.doc2bow(doc) for doc in articles_nouns_and_adjs_bow]

In [ ]:
lda_model_full = models.LdaModel(bow_full, num_topics=4, id2word=dictionary_full, passes=15)
lda_model_nouns = models.LdaModel(bow_nouns, num_topics=4, id2word=dictionary_nouns, passes=15)
lda_model_nouns_and_adjs = models.LdaModel(bow_nouns_and_adjs, num_topics=4, id2word=dictionary_nouns_and_adjs, passes=15)

In [ ]:
X_full_LDA = []
for doc_bow in bow_full:
  document_topics = lda_model_full.get_document_topics(doc_bow, minimum_probability=0.0)
  document_topic_vector = [topic_prob for _, topic_prob in document_topics]
  X_full_LDA.append(document_topic_vector)

X_nouns_LDA = []
for doc_bow in bow_nouns:
  document_topics = lda_model_nouns.get_document_topics(doc_bow, minimum_probability=0.0)
  document_topic_vector = [topic_prob for _, topic_prob in document_topics]
  X_nouns_LDA.append(document_topic_vector)

X_nouns_and_adjs_LDA = []
for doc_bow in bow_nouns_and_adjs:
 document_topics = lda_model_nouns_and_adjs.get_document_topics(doc_bow, minimum_probability=0.0)
 document_topic_vector = [topic_prob for _, topic_prob in document_topics]
 X_nouns_and_adjs_LDA.append(document_topic_vector)

#6 задание

##Support Vector Machines

###TF-IDF

In [ ]:
from sklearn import svm

clf_SVM_TF = svm.SVC()
clf_SVM_TF.fit(X_full_TF[:200].toarray(), articles_target[0][:200])

pred_SVM_TF = []
for i in range(90):
  pred_SVM_TF.append(clf_SVM_TF.predict(X_full_TF[i + 200].toarray())[0])

f_SVM_TF = metrics.f1_score(articles_target[0][200:290], pred_SVM_TF)

###DistilBERT

In [ ]:
clf_SVM_DBERT = svm.SVC()
clf_SVM_DBERT.fit(X_full_DBERT[:200], articles_target[0][:200])

pred_SVM_DBERT = []
for i in range(90):
  pred_SVM_DBERT.append(clf_SVM_DBERT.predict([X_full_DBERT[i + 200]])[0])

f_SVM_DBERT = metrics.f1_score(articles_target[0][200:290], pred_SVM_DBERT)

###LDA

In [ ]:
clf_SVM_LDA = svm.SVC()
clf_SVM_LDA.fit(X_full_LDA[:200], articles_target[0][:200])

pred_SVM_LDA = []
for i in range(90):
  pred_SVM_LDA.append(clf_SVM_LDA.predict([X_full_LDA[i + 200]])[0])

f_SVM_LDA = metrics.f1_score(articles_target[0][200:290], pred_SVM_LDA)

##Naive Bayes

###TF-IDF

In [ ]:
from sklearn.naive_bayes import GaussianNB

clf_NB_TF = GaussianNB()
clf_NB_TF.fit(X_full_TF[:200].toarray(), articles_target[0][:200])

pred_NB_TF = []
for i in range(90):
  pred_NB_TF.append(clf_NB_TF.predict(X_full_TF[i + 200].toarray())[0])

f_NB_TF = metrics.f1_score(articles_target[0][200:290], pred_NB_TF)

###DistilBERT

In [ ]:
clf_NB_DBERT = GaussianNB()
clf_NB_DBERT.fit(X_full_DBERT[:200], articles_target[0][:200])

pred_NB_DBERT = []
for i in range(90):
  pred_NB_DBERT.append(clf_NB_DBERT.predict([X_full_DBERT[i + 200]])[0])

f_NB_DBERT = metrics.f1_score(articles_target[0][200:290], pred_NB_DBERT)

###LDA

In [ ]:
clf_NB_LDA = GaussianNB()
clf_NB_LDA.fit(X_full_LDA[:200], articles_target[0][:200])

pred_NB_LDA = []
for i in range(90):
  pred_NB_LDA.append(clf_NB_LDA.predict([X_full_LDA[i + 200]])[0])

f_NB_LDA = metrics.f1_score(articles_target[0][200:290], pred_NB_LDA)

##Decision Trees

###TF-IDF

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf_DT_TF = DecisionTreeClassifier()
clf_DT_TF.fit(X_full_TF[:200].toarray(), articles_target[0][:200])

pred_DT_TF = []
for i in range(90):
  pred_DT_TF.append(clf_DT_TF.predict(X_full_TF[i + 200].toarray())[0])

f_DT_TF = metrics.f1_score(articles_target[0][200:290], pred_DT_TF)

###DistilBERT

In [ ]:
clf_DT_DBERT = DecisionTreeClassifier()
clf_DT_DBERT.fit(X_full_DBERT[:200], articles_target[0][:200])

pred_DT_DBERT = []
for i in range(90):
  pred_DT_DBERT.append(clf_DT_DBERT.predict([X_full_DBERT[i + 200]])[0])

f_DT_DBERT = metrics.f1_score(articles_target[0][200:290], pred_DT_DBERT)

###LDA

In [ ]:
clf_DT_LDA = DecisionTreeClassifier()
clf_DT_LDA.fit(X_full_LDA[:200], articles_target[0][:200])

pred_DT_LDA = []
for i in range(90):
  pred_DT_LDA.append(clf_DT_LDA.predict([X_full_LDA[i + 200]])[0])

f_DT_LDA = metrics.f1_score(articles_target[0][200:290], pred_DT_LDA)

#7 задание

##Random Forest

###TF-IDF

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_RF_TF = RandomForestClassifier(max_depth=2)
clf_RF_TF.fit(X_full_TF[:200].toarray(), articles_target[0][:200])

pred_RF_TF = []
for i in range(90):
  pred_RF_TF.append(clf_RF_TF.predict(X_full_TF[i + 200].toarray())[0])

f_RF_TF = metrics.f1_score(articles_target[0][200:290], pred_RF_TF)

###DistilBERT

In [ ]:
clf_RF_DBERT = RandomForestClassifier(max_depth=2)
clf_RF_DBERT.fit(X_full_DBERT[:200], articles_target[0][:200])

pred_RF_DBERT = []
for i in range(90):
  pred_RF_DBERT.append(clf_RF_DBERT.predict([X_full_DBERT[i + 200]])[0])

f_RF_DBERT = metrics.f1_score(articles_target[0][200:290], pred_RF_DBERT)

###LDA

In [ ]:
clf_RF_LDA = RandomForestClassifier(max_depth=2)
clf_RF_LDA.fit(X_full_LDA[:200], articles_target[0][:200])

pred_RF_LDA = []
for i in range(90):
  pred_RF_LDA.append(clf_RF_LDA.predict([X_full_LDA[i + 200]])[0])

f_RF_LDA = metrics.f1_score(articles_target[0][200:290], pred_RF_LDA)

##AdaBoost

###TF-IDF

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

clf_AB_TF = AdaBoostClassifier(n_estimators=100, algorithm="SAMME")
clf_AB_TF.fit(X_full_TF[:200].toarray(), articles_target[0][:200])

pred_AB_TF = []
for i in range(90):
  pred_AB_TF.append(clf_AB_TF.predict(X_full_TF[i + 200].toarray())[0])

f_AB_TF = metrics.f1_score(articles_target[0][200:290], pred_AB_TF)

###DistilBERT

In [ ]:
clf_AB_DBERT = AdaBoostClassifier(n_estimators=100, algorithm="SAMME")
clf_AB_DBERT.fit(X_full_DBERT[:200], articles_target[0][:200])

pred_AB_DBERT = []
for i in range(90):
  pred_AB_DBERT.append(clf_AB_DBERT.predict([X_full_DBERT[i + 200]])[0])

f_AB_DBERT = metrics.f1_score(articles_target[0][200:290], pred_AB_DBERT)

###LDA

In [ ]:
clf_AB_LDA = AdaBoostClassifier(n_estimators=100, algorithm="SAMME")
clf_AB_LDA.fit(X_full_LDA[:200], articles_target[0][:200])

pred_AB_LDA = []
for i in range(90):
  pred_AB_LDA.append(clf_AB_LDA.predict([X_full_LDA[i + 200]])[0])

f_AB_LDA = metrics.f1_score(articles_target[0][200:290], pred_AB_LDA)

##Multi-layer Perceptron

###TF-IDF

In [ ]:
from sklearn.neural_network import MLPClassifier

clf_NN_TF = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(50, 20), random_state=1)
clf_NN_TF.fit(X_full_TF[:200].toarray(), articles_target[0][:200])

pred_NN_TF = []
for i in range(90):
  pred_NN_TF.append(clf_NN_TF.predict(X_full_TF[i + 200].toarray())[0])

f_NN_TF = metrics.f1_score(articles_target[0][200:290], pred_NN_TF)

###DistilBERT

In [ ]:
clf_NN_DBERT = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(50, 20), random_state=1)
clf_NN_DBERT.fit(X_full_DBERT[:200], articles_target[0][:200])

pred_NN_DBERT = []
for i in range(90):
  pred_NN_DBERT.append(clf_NN_DBERT.predict([X_full_DBERT[i + 200]])[0])

f_NN_DBERT = metrics.f1_score(articles_target[0][200:290], pred_NN_DBERT)

###LDA

In [ ]:
clf_NN_LDA = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(50, 20), random_state=1)
clf_NN_LDA.fit(X_full_LDA[:200], articles_target[0][:200])

pred_NN_LDA = []
for i in range(90):
  pred_NN_LDA.append(clf_NN_LDA.predict([X_full_LDA[i + 200]])[0])

f_NN_LDA = metrics.f1_score(articles_target[0][200:290], pred_NN_LDA)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


#Итог

In [ ]:
kmeans_metrics = pd.DataFrame({
    "SVM": [f_SVM_TF, f_SVM_DBERT, f_SVM_LDA],
    "Naive Bayes": [f_NB_TF, f_NB_DBERT, f_NB_LDA],
    "Decision Trees": [f_DT_TF, f_DT_DBERT, f_DT_LDA],
    "Random Forest": [f_RF_TF, f_RF_DBERT, f_RF_LDA],
    "Ada Boost": [f_AB_TF, f_AB_DBERT, f_AB_LDA],
    "Perceptron": [f_NN_TF, f_NN_DBERT, f_NN_LDA]
}, index=pd.Index(["TF-IDF", "DistilBert", "LDA"]))

kmeans_metrics.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("F1-score")

,SVM,Naive Bayes,Decision Trees,Random Forest,Ada Boost,Perceptron
TF-IDF,0.954545,0.901099,0.680000,0.767123,0.811321,0.914894
DistilBert,1.000000,0.967033,0.857143,0.988764,0.977778,1.000000
LDA,0.702128,0.687500,0.586957,0.791209,0.739130,0.740000
